# MVP Sprint Engenharia de Dados - Pós Graduação em Ciência de dados e Analytics - PUC-rj 2025
## ---------
## Avaliação de usuários da Netflix

### Objetivo

Neste trabalho, objetivou-se construir um pipeline de dados utilizando databricks que envolveu coleta, carga, modelagem e análise dos dados para obter insights.

Nesse projeto, seguiremos com um **modelo flat table**.

O objetivo aqui proposto é de **estudar o comportamento de diferentes usuários** em uma plataforma de streaming para verificar se diferentes grupos tem diferentes comportamentos.

Para tanto, utilizou-se um dataset obtido no site kaggle (Link: https://pucrio.grupoa.education/plataforma/course/1543924/content/28599812)  que contém 25k amostras de dados fictícios de usuários da netflix para análise, visualização e prática. Incluindo informações demográficas dos usuários, tipo de plano de assinatura, tempo de uso, histórico de login para cada usuário, entre outros dados.

Nesse contexto, o objetivo desta análise é entender:
1. Existe algum plano mais comum entre os usuários avaliados?
2. Existe algum país no qual haja mais usuários?
3. Existe alguma faixa etária mais presente entre os usuários?
4. Como se distribui os usuários em relação ao genêro (masculino x feminino)?
5. Usuários mais jovens tem maior tempo de visualização na plataforma?
6. Usuários mais idosos ficam mais tempo sem logar?
7. Existe algum genero de filme de preferência? Esse genêro muda por país? E por faixa etária? E por sexo?


In [0]:
import pandas as pd
import io
import requests

In [0]:
netflix_users_df = spark.read.option("header", True).option("sep", ",").csv("dbfs:/FileStore/tables/netflix_users.csv")

Observa-se que todos os dados foram importados como texto ('string')

In [0]:
netflix_users_df.show()

+-------+---------------+---+---------+-----------------+----------------+--------------+----------+
|User_ID|           Name|Age|  Country|Subscription_Type|Watch_Time_Hours|Favorite_Genre|Last_Login|
+-------+---------------+---+---------+-----------------+----------------+--------------+----------+
|      1| James Martinez| 18|   France|          Premium|           80.26|         Drama|2024-05-12|
|      2|    John Miller| 23|      USA|          Premium|          321.75|        Sci-Fi|2025-02-05|
|      3|     Emma Davis| 60|       UK|            Basic|           35.89|        Comedy|2025-01-24|
|      4|    Emma Miller| 44|      USA|          Premium|          261.56|   Documentary|2024-03-25|
|      5|     Jane Smith| 68|      USA|         Standard|           909.3|         Drama|2025-01-14|
|      6|  David Johnson| 21|      USA|         Standard|          615.93|       Romance|2025-02-03|
|      7| John Hernandez| 57|   Canada|         Standard|          755.47|       Romance|20

Verificada a tabela, será criada um primeiro stage onde será adicionado esse dataframe AS IS.



In [0]:
%sql DROP DATABASE bronze CASCADE;

In [0]:
%sql CREATE DATABASE bronze;

In [0]:
netflix_users_df.write.mode("overwrite").saveAsTable("bronze.netflix_users")

In [0]:
%sql SELECT * FROM bronze.netflix_users LIMIT 10

User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login
1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12
2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05
3,Emma Davis,60,UK,Basic,35.89,Comedy,2025-01-24
4,Emma Miller,44,USA,Premium,261.56,Documentary,2024-03-25
5,Jane Smith,68,USA,Standard,909.3,Drama,2025-01-14
6,David Johnson,21,USA,Standard,615.93,Romance,2025-02-03
7,John Hernandez,57,Canada,Standard,755.47,Romance,2025-01-05
8,Katie Hernandez,68,USA,Standard,145.23,Sci-Fi,2024-10-30
9,James Williams,39,UK,Basic,950.14,Action,2024-04-16
10,Alex Davis,55,Mexico,Standard,696.66,Horror,2024-07-03


Conforme citado, o dataframe foi importado AS IS como tabela para o stage bronze do pipeline.
Pode-se notar que todas as colunas estão como texto, o que não condiz com a necessidade de análise.

As colunas `age`, `Watch_Time_Hours` precisam ser numéricas para que possam ser usados em cálculos durante a análise.

Já a coluna `Last_Login` é interessante que esteja em formato de data para ser possível fazer quebras temporais.

Portanto, será criado um novo stage no qual esses dados serão tratados e passarão a estar no formato esperado para utilização.


In [0]:
%sql DROP DATABASE silver CASCADE

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2844001472037696>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2844001472037696>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBEQVRBQkFTRSBzaWx2ZXIgQ0FTQ0FERQ==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /d

In [0]:
%sql CREATE DATABASE silver

In [0]:
from pyspark.sql.functions import to_date

netflix_users_df_silver = netflix_users_df.withColumn("Last_Login", to_date(netflix_users_df["Last_Login"], "yyyy-MM-dd"))

In [0]:
from pyspark.sql.functions import col

netflix_users_df_silver = netflix_users_df_silver.withColumn("Age", col("Age").cast("int"))

In [0]:
netflix_users_df_silver = netflix_users_df_silver.withColumn("Watch_Time_Hours", col("Watch_Time_Hours").cast("decimal(10,2)"))

Após as conversões, é importante fazer uma verificação da **qualidade dos dados** na tabela em questão, antes de seguir com a modelagem.
Para isso, é importante avaliar se, agora, os tipos de dados estão de acordo com o esperado, avaliar a distribuição dos mesmos em função do contexto de negócio e se há itens nulos ou fora do esperado no banco de dados.

In [0]:
netflix_users_df_silver.summary().show()

+-------+-----------------+--------------+------------------+---------+-----------------+-----------------+--------------+
|summary|          User_ID|          Name|               Age|  Country|Subscription_Type| Watch_Time_Hours|Favorite_Genre|
+-------+-----------------+--------------+------------------+---------+-----------------+-----------------+--------------+
|  count|            25000|         25000|             25000|    25000|            25000|            25000|         25000|
|   mean|          12500.5|          null|          46.48288|     null|             null|       500.468858|          null|
| stddev|7217.022700994273|          null|19.594861194941107|     null|             null|286.3818152485371|          null|
|    min|                1|    Alex Brown|                13|Australia|            Basic|             0.12|        Action|
|    25%|           6249.0|          null|                29|     null|             null|           256.28|          null|
|    50%|       

- A coluna `user_id` é um valor único que diferencia o usuário (seria nossa chave primária), como não tem duplicatas e não contém nulos, está dentro do esperado
- Existem 25000 nomes na coluna `name` que variam de Alex Brown até Sarah Williams
- Existem 25000 dados de idade no atributo `age` que variam de 13 a 80 anos, com média em 46,5 e desvio padrão em 19,59
- Existem 25000 dados de países na coluna `Country`, 25000 dados de tipo de planos na coluna `Subscription_Type` e 25000 dados de genêro de filme na coluna `Favorite_Genre`
- Exitem 25000 dados de tempo de visualização na coluna `Watch_Time_Hours` que variam de 0,12 a 999,99h com média em 500,47 e desvio padrão em 286,38

In [0]:
from pyspark.sql.functions import col, count, when

netflix_users_df_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in netflix_users_df_silver.columns]).show()

+-------+----+---+-------+-----------------+----------------+--------------+----------+
|User_ID|Name|Age|Country|Subscription_Type|Watch_Time_Hours|Favorite_Genre|Last_Login|
+-------+----+---+-------+-----------------+----------------+--------------+----------+
|      0|   0|  0|      0|                0|               0|             0|         0|
+-------+----+---+-------+-----------------+----------------+--------------+----------+



Não existem itens nulos na tabela utilizada

In [0]:
netflix_users_df_silver.write.mode("overwrite").saveAsTable("silver.netflix_users")

In [0]:
%sql SELECT * FROM silver.netflix_users LIMIT 10

User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login
1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12
2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05
3,Emma Davis,60,UK,Basic,35.89,Comedy,2025-01-24
4,Emma Miller,44,USA,Premium,261.56,Documentary,2024-03-25
5,Jane Smith,68,USA,Standard,909.30,Drama,2025-01-14
6,David Johnson,21,USA,Standard,615.93,Romance,2025-02-03
7,John Hernandez,57,Canada,Standard,755.47,Romance,2025-01-05
8,Katie Hernandez,68,USA,Standard,145.23,Sci-Fi,2024-10-30
9,James Williams,39,UK,Basic,950.14,Action,2024-04-16
10,Alex Davis,55,Mexico,Standard,696.66,Horror,2024-07-03


Na tabela netflix_users no stage silver, os dados estão no formato correto para serem utilizados nas análises pensadas.

Entretanto, para algumas perguntas pensadas, ainda são necessários mais tratamentos. Esses serão avaliados nesta sessão e realizados na modelagem do stage silver para gold.

#### 1. Existe algum plano mais comum entre os usuários avaliados?

In [0]:
%sql SELECT Subscription_Type, COUNT (Name) AS users
FROM silver.netflix_users
GROUP BY Subscription_Type
ORDER BY users DESC

Subscription_Type,users
Premium,8402
Basic,8356
Standard,8242


Os tipos de plano são bem distribuídos entre os usuários avaliados.
O plano mais presente é o Premium, porém com apenas 46 usuários de diferença para o segundo plano mais assinado que é o Basic

#### 2. Existe algum país no _qual_ haja mais usuários?

In [0]:
%sql SELECT Country, COUNT (Name) AS users
FROM silver.netflix_users
GROUP BY Country
ORDER BY users DESC

Country,users
UK,2592
Germany,2547
India,2505
USA,2503
Brazil,2503
Mexico,2493
Canada,2490
France,2473
Japan,2457
Australia,2437


Os usuários também são bem distribuídos entre os países em questão, sendo UK o mais presente, seguido de Alemanha, Índia e USA.

Para responder as perguntas de número 3 a 7 são necessários outros tratamentos dos dados.

Começamos com a verificação da distribuição de idades para criação de faixas etárias

In [0]:
%sql SELECT Age, COUNT (Name) AS users
FROM silver.netflix_users
GROUP BY Age
ORDER BY Age

Age,users
13,339
14,374
15,359
16,373
17,368
18,375
19,357
20,380
21,354
22,368


As idades são bem distribuidas desde 13 anos até 80 anos

Em seguida, verifica-se a última data presente na tabela para criar uma coluna que faça a diferença entre o último login e o último dia avaliado, criando um tempo desde o último login

In [0]:
%sql SELECT MAX(Last_login) 
FROM silver.netflix_users

max(Last_login)
2025-03-08


A última data de login foi no dia 08-03-2025

Com essas informações, faremos a consulta para criar uma coluna denominada `Not_Logged_Days` que faz a diferença entre o último dia avaliado e o último dia de login com o objetivo de entender à quantos dias o usuário não acessa.

Nessa consulta, também criamos uma coluna `Age_Groups` onde separamos as faixas etárias a serem avaliadas.

In [0]:
%sql SELECT *,
DATEDIFF('2025-03-08', Last_Login) as Not_Logged_Days,
CASE 
  WHEN Age < 18 THEN 'Teenager'
  WHEN Age >= 18 and Age <= 28 THEN 'Young-Adult'
  WHEN Age > 28 and Age <= 39 THEN 'Adult'
  WHEN Age > 39 and Age <= 64 THEN 'Middle-Aged'
  WHEN Age > 64 THEN 'Senior'
END AS Age_Groups
FROM silver.netflix_users
LIMIT 10

User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login,Not_Logged_Days,Age_Groups
1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12,300,Young-Adult
2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05,31,Young-Adult
3,Emma Davis,60,UK,Basic,35.89,Comedy,2025-01-24,43,Middle-Aged
4,Emma Miller,44,USA,Premium,261.56,Documentary,2024-03-25,348,Middle-Aged
5,Jane Smith,68,USA,Standard,909.30,Drama,2025-01-14,53,Senior
6,David Johnson,21,USA,Standard,615.93,Romance,2025-02-03,33,Young-Adult
7,John Hernandez,57,Canada,Standard,755.47,Romance,2025-01-05,62,Middle-Aged
8,Katie Hernandez,68,USA,Standard,145.23,Sci-Fi,2024-10-30,129,Senior
9,James Williams,39,UK,Basic,950.14,Action,2024-04-16,326,Adult
10,Alex Davis,55,Mexico,Standard,696.66,Horror,2024-07-03,248,Middle-Aged


Para separação entre grupos separando os usuários por masculino e feminino, será feito uma tentativa de avaliar o primeiro nome e verificar se o mesmo é masculino ou feminino.

In [0]:
%sql WITH names AS (
  SELECT 
    SPLIT_PART(Name, ' ', 1) AS Fist_name,
    SPLIT_PART(Name, ' ', 2) AS Last_name
  FROM silver.netflix_users
)

SELECT DISTINCT Fist_name FROM names;

Fist_name
James
Chris
Katie
Michael
Sarah
Alex
John
Emma
Jane
David


Como existem apenas 10 nomes distintos, é possível fazer a separação atráves de grupos criados manualmente. Para tanto, partiremos da premissa que os nomes: 'James', 'Chris', 'Michael', 'John', 'David' são masculinos e os nomes 'Katie', 'Sarah', 'Alex', 'Emma', 'Jane' são femininos.

In [0]:
%sql SELECT *,
DATEDIFF('2025-03-08', Last_Login) as Not_Logged_Days,
CASE 
  WHEN Age < 18 THEN 'Teenager'
  WHEN Age >= 18 and Age <= 28 THEN 'Young-Adult'
  WHEN Age > 28 and Age <= 39 THEN 'Adult'
  WHEN Age > 39 and Age <= 64 THEN 'Middle-Aged'
  WHEN Age > 64 THEN 'Senior'
END AS Age_Groups,
CASE
  WHEN SPLIT_PART(Name, ' ', 1) in ('James', 'Chris', 'Michael', 'John', 'David') THEN 'Male'
  WHEN SPLIT_PART(Name, ' ', 1) in ('Katie', 'Sarah', 'Alex', 'Emma', 'Jane') THEN 'Female'
END AS Gender

FROM silver.netflix_users
LIMIT 10

User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login,Not_Logged_Days,Age_Groups,Gender
1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12,300,Young-Adult,Male
2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05,31,Young-Adult,Male
3,Emma Davis,60,UK,Basic,35.89,Comedy,2025-01-24,43,Middle-Aged,Female
4,Emma Miller,44,USA,Premium,261.56,Documentary,2024-03-25,348,Middle-Aged,Female
5,Jane Smith,68,USA,Standard,909.30,Drama,2025-01-14,53,Senior,Female
6,David Johnson,21,USA,Standard,615.93,Romance,2025-02-03,33,Young-Adult,Male
7,John Hernandez,57,Canada,Standard,755.47,Romance,2025-01-05,62,Middle-Aged,Male
8,Katie Hernandez,68,USA,Standard,145.23,Sci-Fi,2024-10-30,129,Senior,Female
9,James Williams,39,UK,Basic,950.14,Action,2024-04-16,326,Adult,Male
10,Alex Davis,55,Mexico,Standard,696.66,Horror,2024-07-03,248,Middle-Aged,Female


Agora, será criado o stage gold, com os dados modelados através da consulta testada, de modo a facilitar a análise utilizando grupos de genero e faixa etária, assim como o cálculo de dias não logados.

In [0]:
%sql DROP DATABASE gold CASCADE

In [0]:
%sql CREATE DATABASE gold

In [0]:
%sql CREATE TABLE gold.netflix_users AS
SELECT *,
DATEDIFF('2025-03-08', Last_Login) as Not_Logged_Days,
CASE 
  WHEN Age < 18 THEN 'Teenager'
  WHEN Age >= 18 and Age <= 28 THEN 'Young-Adult'
  WHEN Age > 28 and Age <= 39 THEN 'Adult'
  WHEN Age > 39 and Age <= 64 THEN 'Middle-Aged'
  WHEN Age > 64 THEN 'Senior'
END AS Age_Groups,
CASE
  WHEN SPLIT_PART(Name, ' ', 1) in ('James', 'Chris', 'Michael', 'John', 'David') THEN 'Male'
  WHEN SPLIT_PART(Name, ' ', 1) in ('Katie', 'Sarah', 'Alex', 'Emma', 'Jane') THEN 'Female'
END AS Gender

FROM silver.netflix_users

num_affected_rows,num_inserted_rows


In [0]:
%sql SELECT * FROM gold.netflix_users
LIMIT 10

User_ID,Name,Age,Country,Subscription_Type,Watch_Time_Hours,Favorite_Genre,Last_Login,Not_Logged_Days,Age_Groups,Gender
1,James Martinez,18,France,Premium,80.26,Drama,2024-05-12,300,Young-Adult,Male
2,John Miller,23,USA,Premium,321.75,Sci-Fi,2025-02-05,31,Young-Adult,Male
3,Emma Davis,60,UK,Basic,35.89,Comedy,2025-01-24,43,Middle-Aged,Female
4,Emma Miller,44,USA,Premium,261.56,Documentary,2024-03-25,348,Middle-Aged,Female
5,Jane Smith,68,USA,Standard,909.30,Drama,2025-01-14,53,Senior,Female
6,David Johnson,21,USA,Standard,615.93,Romance,2025-02-03,33,Young-Adult,Male
7,John Hernandez,57,Canada,Standard,755.47,Romance,2025-01-05,62,Middle-Aged,Male
8,Katie Hernandez,68,USA,Standard,145.23,Sci-Fi,2024-10-30,129,Senior,Female
9,James Williams,39,UK,Basic,950.14,Action,2024-04-16,326,Adult,Male
10,Alex Davis,55,Mexico,Standard,696.66,Horror,2024-07-03,248,Middle-Aged,Female


#### Catálogo de dados tabela gold.netflix_users

###### - `User_ID` - Identificador único para cada usuário (PK)
    string;
    deve conter apenas dados únicos;
    não deve conter nulos;
    faixa: [1 - 25000];

###### - `Name` - Nome do usuário
    string;
    deve contar apenas dados únicos;
    não deve conter nulos;

###### - `Age` - Idade do usuário
    integer;
    não deve conter nulos;
    faixa: [13 - 80];

###### - `Country` - País de residência do usuário
    string;
    não deve conter nulos;

###### - `Subscription_Type` - Plano contratado pelo usuário
    string;
    não deve conter nulos;
    categórico: [Basic, Standard, Premium];

###### - `Watch_Time_Hours` - Tempo de utilização da plataforma pelo usuário
    decimal;
    pode conter nulos;
    faixa: [0-999,9];

###### - `Favorite_genre` - Genêro favorito do usuário
    string;
    não pode conter nulos;
    categórico: [Horror, Documentary, Action, Romance, Comedy]

###### - `Last_Login` - Data na qual o usuário fez seu último login na plataforma
    date;
    não pode conter nulos;

###### - `Not_Logged_Days` - Dias desde o último login do usuário na plataforma
    integer;
    não pode conter nulos;
    modelado;

###### - `Age_groups` - Faixa etária do usuário
    string;
    não pode conter nulos;
    modelado;
    categórico: [Teenager, Young-Adult, Adult, Middle-Aged, Senior]

###### - `Gender` - Genêro do usuário
    string;
    não pode conter nulos;
    modelado;
    categórico: [Male, Female]





Agora, seguimos para criar consultas de análise de dados, de modo a responder as demais perguntas realizadas

#### 3. Existe alguma faixa etária mais presente entre os usuários?![](path)

In [0]:
%sql WITH base AS (
  SELECT Age_groups, COUNT (Name) AS users
  FROM gold.netflix_users
  GROUP BY Age_groups
  ORDER BY users DESC
)
SELECT Age_groups, users, (users / (SELECT SUM(users) FROM base) * 100) AS share
FROM base
GROUP BY Age_groups, users

Age_groups,users,share
Middle-Aged,9304,37.216
Senior,5822,23.288
Young-Adult,4104,16.416
Adult,3957,15.828000000000001
Teenager,1813,7.252


Observa-se uma maior quantidade de usuários na categoria Adulto de meia idade, representando 37,216% da base de usuários, enquanto que o menor grupo é de adolescentes, representando 7,252% da base

#### 4. Como se distribui os usuários em relação ao genêro (masculino x feminino)?

In [0]:
%sql WITH base AS (
  SELECT Gender, COUNT (Name) AS users
  FROM gold.netflix_users
  GROUP BY Gender
  ORDER BY users DESC
)
SELECT Gender, users, (users / (SELECT SUM(users) FROM base) * 100) AS share
FROM base
GROUP BY Gender, users

Gender,users,share
Female,12589,50.356
Male,12411,49.644


A base é bem distribuída entre usuários do sexo masculino (49,64%) e feminino (50,36%)

#### 5. Usuários mais jovens tem maior tempo de visualização na plataforma?

In [0]:
%sql SELECT Age_groups, AVG (Watch_Time_Hours) AS avg_watch_Time_Hours
FROM gold.netflix_users
GROUP BY Age_groups
ORDER BY avg_watch_Time_Hours DESC

Age_groups,avg_watch_Time_Hours
Adult,508.816053
Senior,505.454686
Middle-Aged,497.751509
Young-Adult,496.026545
Teenager,490.240579


Invalidando a hipótese da pergunta, na realidade, usuários adolecentes tem a menor média de tempo na plataforma dentre os diferentes grupos etários, ficando 490,24h em média na plataforma. Seguido pelo jovem adulto.

Os usuários de maior tempo médio são adultos com em média 508,8h e idosos com 505,5h.

#### 6. Usuários mais idosos ficam mais tempo sem logar?


In [0]:
%sql SELECT Age_groups, AVG (Not_Logged_Days) AS avg_Not_Logged_Days
FROM gold.netflix_users
GROUP BY Age_groups
ORDER BY avg_Not_Logged_Days DESC

Age_groups,avg_Not_Logged_Days
Young-Adult,184.33771929824562
Teenager,183.59073359073358
Middle-Aged,182.26730438521065
Adult,180.2203689663887
Senior,179.96873926485745


Também invalidando a hipótese da pergunta, idosos são, na verdade aqueles que estão, em média, a menos tempo sem logar (179,97h)

#### 7. Existe algum genero de filme de preferência? Esse genêro muda por país? E por faixa etária? E por sexo?

In [0]:
%sql SELECT Favorite_genre, COUNT (Name) AS users
FROM gold.netflix_users
GROUP BY Favorite_genre
ORDER BY users DESC

Favorite_genre,users
Horror,3654
Documentary,3636
Action,3589
Romance,3572
Comedy,3561
Drama,3533
Sci-Fi,3455


Embora o genêro Horror seja mais presente, a distribuição entre os diferentes genêros na base avaliada é muito próxima.

Observaremos a base verificando por sexo do usuário e país:

In [0]:
%sql SELECT Favorite_genre, Country, COUNT (Name) AS users
FROM gold.netflix_users
GROUP BY Country, Favorite_genre
ORDER BY Country, users DESC

Favorite_genre,Country,users
Documentary,Australia,368
Romance,Australia,358
Sci-Fi,Australia,352
Horror,Australia,351
Drama,Australia,348
Action,Australia,330
Comedy,Australia,330
Romance,Brazil,376
Comedy,Brazil,373
Action,Brazil,364


Embora os diferentes generos favoritos sejam bem distribuídos, existe sim uma leve diferenciação por país entre os diferentes genêros preferidos entre os usuários avaliados, por exemplo:

No Brasile e India, o genero favorito é romance, enquanto que na Austrália, França, Japão, México e USA é Documentário, já no Canadá é Ação, na Alemanha Sci-Fi e no UK Horror.

In [0]:
%sql SELECT Favorite_genre, Age_Groups, COUNT (Name) AS users
FROM gold.netflix_users
GROUP BY Age_Groups, Favorite_genre
ORDER BY Age_Groups, users DESC

Favorite_genre,Age_Groups,users
Sci-Fi,Adult,603
Romance,Adult,590
Documentary,Adult,584
Horror,Adult,571
Action,Adult,562
Comedy,Adult,536
Drama,Adult,511
Horror,Middle-Aged,1389
Drama,Middle-Aged,1348
Documentary,Middle-Aged,1342


Também existe uma diferença de genêro preferido nas diferentes faixas etárias (reforçando que dentro da mesma faixa etária, a distribuição é muito próxima)

Adultos tem um share maior na categoria Sci-Fi,  Jovens adultos e adultos de meia idade preferem Horror, enquanto idosos preferem documentários e adolescentes comédia.

In [0]:
%sql SELECT Favorite_genre, Gender, COUNT (Name) AS users
FROM gold.netflix_users
GROUP BY Gender, Favorite_genre
ORDER BY Gender, users DESC

Favorite_genre,Gender,users
Horror,Female,1821
Documentary,Female,1818
Action,Female,1818
Comedy,Female,1813
Drama,Female,1802
Romance,Female,1783
Sci-Fi,Female,1734
Horror,Male,1833
Documentary,Male,1818
Romance,Male,1789


A distribuição entre genêros é muito próxima para ambos os sexos, sendo Horror e Documentário os favoritos em ambos e casos e Sci-Fi o menos adorado em ambos os casos.

Adicionando uma nova pergunta à análise: Usuários que preferem um determino genero de filme tem mais tempo de tela?

In [0]:
%sql SELECT Favorite_genre, AVG (Watch_Time_Hours) AS avg_watch_Time_Hours
FROM gold.netflix_users
GROUP BY Favorite_genre
ORDER BY avg_watch_Time_Hours DESC

Favorite_genre,avg_watch_Time_Hours
Romance,506.899950
Drama,504.482423
Comedy,501.868756
Documentary,500.694868
Action,498.495163
Sci-Fi,496.379340
Horror,494.517649


Usuários que gostam de Romance são os mais presentes na plataforma, com uma média de 506,9h de tempo de visualização, enquanto àqueles que gostam de horror tem menor resultado, ficando em média 494,5h na plataforma.

Com isso, todas as perguntas foram respondidas, incluindo uma variação da pergunta 7 que surgiu ao longo do caminho.

As hipóteses testadas nas perguntas 5 e 6 foram invalidadas.

#### Autoavaliação

Iniciei o MVP sem muita direção do que deveria fazer, as aulas me ajudaram bastante no direcionamento, assim com as respostas no discord. Com isso, consegui entender bem o objetivo do trabalho e a forma esperada de execução.

Como trabalho em um marketplace e estudo bastante comportamente de usuário no plataforma, achei que seria interessante olhar esse cenário no MVP utilizando dados de utilização da netflix, tema de meu interesse. Por isso, busquei datasets nesse contexto.

No decorrer do trabaho, percebi que o dataset escolhido, por ser fictíccio era muito simples e com uma distribuição muito similar entre grupos, o que tornou o trabalho um pouco menos interessante quando o objetivo era insights sobre o comportamento do usuário da netflix.

A pergunta voltada para genero dos usuários não seria possível de ser respondida em um banco mais variado, uma vez que exisitiriam mais de 10 primeiros nomes, a classificação se tornaria inviável, portanto, essa foi uma pergunta que só foi respondida devido a simplicidade dos dados.

O dataset escolhido também facilitou o trabalho de qualidade de dados, uma vez que não havia dados fora do padrão, outliers ou nulos.

Um outro ponto negativo desse dataset é que ele já contém alguns dados agregados, por exemplo o tempo de visualização. Em um novo cenário para insights mais detalhados seria interessante ter uma tabela de usuários, com User_Id [PK] e atributos desse usuário (por exemplo, o genero poderia estar aqui, de modo a não ser necessária a separação por primeiro nome) e uma tabela de sessões (Sesion_ID (PK) e User_ID (FK)) onde teríamos o tempo de acesso e cada genêro acessado, não somente o genero favorito. Esse tipo de estrutura tornaria a modelagem mais rica e os resultados das análises também.

Apesar dos pontos de atenção citados, foi possível executar o pipeline de dados em nuvem e criar uma modelagem de flat table que facilitasse as análises e as perguntas propostas foram respondidas utilizando SQL de forma simples e direta.

Dessa forma, considero que o projeto e o relatório cumprem os requisitos e o aprendizado foi de grande valia para a minha carreira, uma vez que nunca  tinha atuado com o databricks e criação de pipelines de dados.
